# PoseView: Molecular Interaction Patterns at a Glance
This notebook will introduce the proteins.plus API endpoints for Poseview. In general we will need a protein and a ligand to generate 2D projections of protein-ligand interactions. The ligand does not need to be the native ligand and can be anywhere relative to the protein as long as it is meaningful to you.

*Katrin Stierand and Matthias Rarey
Journal of Cheminformatics 2010, 2, No. 1 p. P50
[https://doi.org/10.1186/1758-2946-2-S1-P50](https://doi.org/10.1186/1758-2946-2-S1-P50)*

In [1]:
# imports
import io
from pathlib import Path
import requests
import sys
import time
from urllib.parse import urljoin

from IPython.display import Image
from Bio.PDB import PDBParser
import nglview as nv
from rdkit import Chem

In [2]:
# constants
TEST_FILES = Path('../test_files/')
PROTEINS_PLUS_URL = 'http://localhost:8000/'
UPLOAD = PROTEINS_PLUS_URL + 'molecule_handler/upload/'
UPLOAD_JOBS = PROTEINS_PLUS_URL + 'molecule_handler/upload/jobs/'
PROTEINS = PROTEINS_PLUS_URL + 'molecule_handler/proteins/'
LIGANDS = PROTEINS_PLUS_URL + 'molecule_handler/ligands/'
POSEVIEW = urljoin(PROTEINS_PLUS_URL, 'poseview/')
POSEVIEW_JOBS = urljoin(PROTEINS_PLUS_URL, 'poseview/jobs/')

In [3]:
# utils

# check server connection
try:
    response = requests.get(PROTEINS_PLUS_URL)
except requests.ConnectionError as error:
    if 'Connection refused' in str(error):
        print('WARNING: could not establish a connection to the server', file=sys.stderr)
    raise
    
def poll_job(job_id, poll_url, poll_interval=1, max_polls=10):
    """Poll the progress of a job
    
    Continuosly polls the server in regular intervals and updates the job information, especially the status.
    
    :param job_id: UUID of the job to poll
    :type job_id: str
    :param poll_url: URl to send the polling request to
    :type poll_url: str
    :param poll_interval: time interval between polls in seconds
    :type poll_interval: int
    :param max_polls: maximum number of times to poll before exiting
    :type max_polls: int
    :return: polled job
    :rtype: dict
    """
    job = requests.get(poll_url + job_id).json()
    status = job['status']
    current_poll = 0
    while status == 'pending' or status == 'running':
        print(f'Job {job_id} is { status }')
        current_poll += 1
        if current_poll >= max_polls:
            print(f'Job {job_id} has not completed after {max_polls} polling requests' \
                  f' and {poll_interval * max_polls} seconds')
            return job
        time.sleep(poll_interval)
        job = requests.get(poll_url + job_id).json()
        status = job['status']
    print(f'Job {job_id} completed with { status }')
    return job

To get a PoseView picture of your binding site you can upload the protein and the ligand defining your binding site:

In [4]:
with open(TEST_FILES / 'NXG_A_1294.sdf') as upload_ligand_file:
    with open(TEST_FILES / '4agn.pdb') as upload_file:
        query = {'protein_file': upload_file, 'ligand_file': upload_ligand_file}
        job_submission = requests.post(POSEVIEW, files=query).json()
poseview_job = poll_job(job_submission['job_id'], POSEVIEW_JOBS)
print(poseview_job['image'])
Image(url=poseview_job['image'], width=600, height=600)  # freely scalabe SVG

Job 2f20e66f-f40f-4f29-9a2b-cbdc337a5cb7 completed with success
http://localhost:8000/media/poseview/tmp7kl7m3o6.svg


PoseView images are freely scalable and editable SVGs. The PoseView job model contains a URL to the actual image. If you want to keep the image please download the image (ex. you could "right click" -> "save target as" the URL above).

You can also use the preprocessor to split ligands from an uploaded protein file and then use one of them to generate a PoseView image with. Here we will upload the same file but to the preprocessor:

In [5]:
with open(TEST_FILES / '4agn.pdb') as upload_file:
    query = {'protein_file': upload_file}
    preprocessing_job_submission = requests.post(UPLOAD, files=query).json()
preprocessing_job = poll_job(preprocessing_job_submission['job_id'], UPLOAD_JOBS)

Job 297a31fc-b3ef-4347-9a73-3676b01aaf20 completed with success


Then instead of taking the ligand of the A chain we will take the ligand of the B chain and generate a PoseView image for it:

In [6]:
protein = requests.get(PROTEINS + preprocessing_job['output_protein']).json()
ligands = [requests.get(LIGANDS + ligand_id).json() for ligand_id in protein['ligand_set']]
for ligand in ligands:
    if ligand['name'] == 'NXG_B_1292':  # get the ligand of the B chain
        other_ligand = ligand
        break

query = {'protein_id': protein['id'], 'ligand_id': ligand['id']}
job_submission = requests.post(POSEVIEW, data=query).json()
poseview_job = poll_job(job_submission['job_id'], POSEVIEW_JOBS)
print(poseview_job['image'])
Image(url=poseview_job['image'], width=600, height=600)  # freely scalabe SVG

Job f55040ca-a38c-4780-8fe6-4427b439144b completed with success
http://localhost:8000/media/poseview/tmp8yqy2i62.svg


The image is nearly the same but this time with references to B chain residues.

Another thing we can do is generate PoseView images for ligands that are non-native to the protein. For that we will extract a ligand from PDB code: 4AGO and put it in our preprocessed 4AGM entry.

In [7]:
# preprocess 4AGO
query = {'pdb_code': '4ago'}
preprocessing_job_submission = requests.post(UPLOAD, data=query).json()
preprocessing_job = poll_job(preprocessing_job_submission['job_id'], UPLOAD_JOBS)
other_protein = requests.get(PROTEINS + preprocessing_job['output_protein']).json()
other_ligands = [requests.get(LIGANDS + ligand_id).json() for ligand_id in other_protein['ligand_set']]

# submit the preprocessed 4AGM from before and a ligand from 4AGO
query = {'protein_id': protein['id'], 'ligand_id': other_ligands[0]['id']}
job_submission = requests.post(POSEVIEW, data=query).json()
poseview_job = poll_job(job_submission['job_id'], POSEVIEW_JOBS)
print(poseview_job['image'])
Image(url=poseview_job['image'], width=600, height=600)  # freely scalabe SVG

Job 6e3b7bab-b3e9-4ad8-8a79-1ff398d3037e completed with success
Job b2f11e52-6a46-44c3-8f61-4c4030227232 completed with success
http://localhost:8000/media/poseview/tmpfnfo6818.svg


The above picture is of P74 from 4AGO in 4AGM. You could do similar operations with, for example, poses from a docking.